[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/advanced/04_Conflict_Resolution_Strategies.ipynb)

# Conflict Resolution Strategies

## Overview

Detect conflicts in knowledge graphs, apply multiple resolution strategies, track sources, and maintain audit trails.


**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/conflicts/)

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```

## Workflow: Detect Conflicts → Multiple Resolution Strategies → Track Sources → Audit


In [ ]:
%pip install -U "semantica[all]"
import semantica
print(semantica.__version__)


In [ ]:
from datetime import datetime
import json
from semantica.conflicts import ConflictDetector, ConflictResolver, SourceTracker
from semantica.conflicts.conflict_resolver import ResolutionStrategy

## Step 1: Define Entities with Conflicting Data


In [ ]:
entities = [
    {
        "id": "e1",
        "type": "Person",
        "name": "John Doe",
        "age": 30,
        "location": "New York",
        "source": "source1",
        "confidence": 0.8,
        "metadata": {"timestamp": datetime(2023, 1, 1)}
    },
    {
        "id": "e1",
        "type": "Person",
        "name": "John Doe",
        "age": 32,
        "location": "Boston",
        "source": "source2",
        "confidence": 0.9,
        "metadata": {"timestamp": datetime(2023, 6, 1)}
    },
    {
        "id": "e2",
        "type": "Organization",
        "name": "Tech Corp",
        "founded": 2010,
        "employees": 100,
        "source": "source1",
        "confidence": 0.9,
        "metadata": {"timestamp": datetime(2023, 1, 1)}
    },
    {
        "id": "e2",
        "type": "Organization",
        "name": "Tech Corp",
        "founded": 2012,
        "employees": 150,
        "source": "source2",
        "confidence": 0.7,
        "metadata": {"timestamp": datetime(2023, 3, 1)}
    }
]

## Step 2: Detect Conflicts


In [ ]:
# Initialize detector
detector = ConflictDetector(track_provenance=True)

# Detect conflicts across all properties
conflicts = detector.detect_entity_conflicts(entities)

print(f"Detected {len(conflicts)} conflicts:")
for i, conflict in enumerate(conflicts, 1):
    print(f"\nConflict {i}:")
    print(f"  ID: {conflict.conflict_id}")
    print(f"  Type: {conflict.conflict_type.value}")
    print(f"  Entity: {conflict.entity_id}")
    print(f"  Property: {conflict.property_name}")
    print(f"  Values: {conflict.conflicting_values}")
    print(f"  Severity: {conflict.severity}")

## Step 3: Resolve Conflicts

We can apply different strategies to resolve the conflicts:
- **Voting**: Selects the most frequent value
- **Most Recent**: Selects the value with the latest timestamp
- **Highest Confidence**: Selects the value from the source with highest confidence


In [ ]:
# Initialize resolver
resolver = ConflictResolver()

# Strategy 1: Voting
print("--- Strategy: Voting ---")
results_voting = resolver.resolve_conflicts(conflicts, strategy="voting")
for r in results_voting:
    if r.resolved:
        print(f"Resolved {r.conflict_id}: {r.resolved_value} (Confidence: {r.confidence:.2f})")

# Strategy 2: Most Recent
print("\n--- Strategy: Most Recent ---")
results_recent = resolver.resolve_conflicts(conflicts, strategy="most_recent")
for r in results_recent:
    if r.resolved:
        print(f"Resolved {r.conflict_id}: {r.resolved_value}")

# Strategy 3: Highest Confidence
print("\n--- Strategy: Highest Confidence ---")
results_confidence = resolver.resolve_conflicts(conflicts, strategy="highest_confidence")
for r in results_confidence:
    if r.resolved:
        print(f"Resolved {r.conflict_id}: {r.resolved_value} (Confidence: {r.confidence:.2f})")

## Step 4: Track Sources

The `ConflictDetector` tracks source provenance when `track_provenance=True`.

In [ ]:
tracker = detector.source_tracker

for conflict in conflicts:
    print(f"\nConflict: {conflict.conflict_id}")
    # Get detailed source info for the property
    sources = tracker.get_property_sources(conflict.entity_id, conflict.property_name)
    if sources:
        print(f"  Entity: {conflict.entity_id}, Property: {conflict.property_name}")
        print(f"  Sources found: {len(sources.sources)}")
        for src in sources.sources:
            print(f"    - {src.document} (Confidence: {src.confidence})")

## Step 5: Audit Trail

The `ConflictResolver` maintains a history of all resolutions.

In [ ]:
history = resolver.get_resolution_history()

print(f"Resolution History ({len(history)} entries):")
for entry in history:
    print(f"\nConflict: {entry.conflict_id}")
    print(f"  Strategy: {entry.resolution_strategy}")
    print(f"  Resolved Value: {entry.resolved_value}")
    print(f"  Notes: {entry.resolution_notes}")

## Summary

Conflict resolution workflow:
- Conflict Detection using `ConflictDetector`
- Multiple Resolution Strategies (Voting, Most Recent, Highest Confidence)
- Source Tracking with `SourceTracker`
- Complete Audit Trail via `ConflictResolver`